### INF 510 FINAL PROJECT
##### Name: Peter Argo
##### submit: 12/13/19

Overview:
<br>
The goal is to evaluate which MLB regular season team statistics predicted success of teams in the postseason
<br>
This notebook shall scrape data from the following 3 sources:
<br>
1. Baseball reference
<br>
2. Fox Sports
<br>
3. MLB API
<br>
This notebook shall format the data using SQL and Pandas as primary methods
<br>
This Notbook shall analyze the data using a logistic regression to draw conclusions

First we need to import all modules used to do our analysis. Importing MLB_functions initializes the function contained within and creates our database and tables used to store data. 
<br>
Important note: If you want to re-initialize the tables, you must restart the kernal and import the MLB_functions again as this is where the database is orginally generated

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import sys
sys.path.insert(0, os.path.abspath("C:\\Users\\Peter Argo\\Desktop\\INF 510\\Project\\src"))
from MLB_functions import *
import MLB_functions
import requests
import re
import time
import csv
import collections
import pprint
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , confusion_matrix

import matplotlib.pyplot as plt
from sklearn import svm, datasets
# from sklearn.metrics import plot_confusion_matrix

### Section 1: Extract post season series from Baseball Reference

function baseball_ref_scraper gets the data from baseball reference
the data are arrays containing the year the winner, the loser, and the title of each series as well as a booloen 
determining if the team with more reg season wins won and a cnt value of times a series was played by team with 
the same number of regular season wins.
The value input into baseball_ref_scraper determines the number of post season series that we want to analyse
value must be less or equal to than 190

***READ THIS***
<br>
This will be more clear as you walk through the notebook, however I reccomend inputing 190 for the number_of_series (maximum possible value.  This will show its robustness. Then execute every cell until you get to Section 3 (API data  extract section). Skip Section 3. Then run all cells in section 4 (Analysis 1 of 2). This will provide the logic used throughout using the largest dataset possible.
<br>
<br>
Subsequently, restart the kernal. Import the above module to reintialize our database. Then Run through the entire notbook as layed out however I would not input a number greater than 50 for number_of_series. The API scraper requires much processing and thus larger numbers take significantly more time to scrape for the data.

In [2]:
number_of_series = int(input("input a value below 191:"))
while number_of_series > 190:
    number_of_series = int(input("please provide a value lower than 191. Thanks!:"))
print(f"you chose: {number_of_series}")

input a value below 191:50
you chose: 50


In [3]:
(year, winning_team_array, losing_team_array, postseason_series, more_wins, cnt) = baseball_ref_scraper(number_of_series)

# print valuse to visualize output if needed

print(winning_team_array)
print('\n')
print(losing_team_array)
print('\n')
print(postseason_series)
print('\n')
print(year)
print('\n')
print(more_wins)
print('\n')
print(cnt)

['Nationals', 'Astros', 'Nationals', 'Astros', 'Yankees', 'Nationals', 'Cardinals', 'Rays', 'Nationals', 'Red Sox', 'Red Sox', 'Dodgers', 'Red Sox', 'Astros', 'Brewers', 'Dodgers', 'Yankees', 'Rockies', 'Astros', 'Astros', 'Dodgers', 'Yankees', 'Astros', 'Dodgers', 'Cubs', 'Yankees', 'Diamondbacks', 'Cubs', 'Indians', 'Cubs', 'Blue Jays', 'Indians', 'Cubs', 'Dodgers', 'Blue Jays', 'Giants', 'Royals', 'Royals', 'Mets', 'Royals', 'Blue Jays', 'Cubs', 'Mets', 'Astros', 'Cubs', 'Giants', 'Royals', 'Giants', 'Royals', 'Orioles']


['Astros', 'Yankees', 'Cardinals', 'Rays', 'Twins', 'Dodgers', 'Braves', 'Athletics', 'Brewers', 'Dodgers', 'Astros', 'Brewers', 'Yankees', 'Indians', 'Rockies', 'Braves', 'Athletics', 'Cubs', 'Dodgers', 'Yankees', 'Cubs', 'Indians', 'Red Sox', 'Diamondbacks', 'Nationals', 'Twins', 'Rockies', 'Indians', 'Blue Jays', 'Dodgers', 'Rangers', 'Red Sox', 'Giants', 'Nationals', 'Orioles', 'Mets', 'Mets', 'Blue Jays', 'Cubs', 'Astros', 'Rangers', 'Cardinals', 'Dodgers', '

The number of times the team with the winning record wins their postseason series is determined to be approx 50% if the time. The number of times there were equal wins between the competing times was not considered in this calculation. Therefore if you be bet squarely on number of wins a team has vs the opposing team, you could expect to win approximatly half the time. Using wins as our target, we want to improve on these odds.

In [4]:
counter=collections.Counter(more_wins)
print(f"the percent of series where the team with the better record advanced is {counter[1]/(counter[0]+counter[1]-cnt)*100}% for this sample")

the percent of series where the team with the better record advanced is 43.75% for this sample


Create the Primary Key database
<br>
Requires input .csv file from Data subdirectory

In [2]:
val_prim = []
with open('..\data\MLB teams.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for line in csv_reader:
        line[0] = int(line[0])
        line[3] = int(line[3])
        val_prim.append((line[0],str(line[1]),str(line[2]),line[3]))

sql_prim = 'INSERT INTO prim_ID (team_id, team_city, team_name, API_ID) VALUES (?, ?, ?, ?)'

cur.executemany(sql_prim,val_prim)

cur.execute('SELECT * FROM prim_ID')
results = cur.fetchall()
print(results)

[(1, 'Arizona', 'Diamondbacks', 109), (2, 'Atlanta', 'Braves', 130), (3, 'Baltimore', 'Orioles', 110), (4, 'Boston', 'Red Sox', 111), (5, 'Chicago_', 'Cubs', 112), (6, 'Chicago', 'White Sox', 145), (7, 'Cincinnati', 'Reds', 113), (8, 'Cleveland', 'Indians', 114), (9, 'Colorado', 'Rockies', 115), (10, 'Detroit', 'Tigers', 116), (11, 'Houston', 'Astros', 117), (12, 'Kansas City', 'Royals', 118), (13, 'Los Angeles', 'Angels', 108), (14, 'Los Angeles_', 'Dodgers', 119), (15, 'Miami', 'Marlins', 146), (16, 'Milwaukee', 'Brewers', 158), (17, 'Minnesota', 'Twins', 142), (18, 'New York', 'Mets', 121), (19, 'New York_', 'Yankees', 147), (20, 'Oakland', 'Athletics', 133), (21, 'Philadelphia', 'Phillies', 143), (22, 'Pittsburgh', 'Pirates', 134), (23, 'San Diego', 'Padres', 135), (24, 'San Francisco', 'Giants', 137), (25, 'Seattle', 'Mariners', 136), (26, 'St. Louis', 'Cardinals', 138), (27, 'Tampa Bay', 'Rays', 139), (28, 'Texas', 'Rangers', 140), (29, 'Toronto', 'Blue Jays', 141), (30, 'Washing

Creater Database for each postseason series winning team and losing team
(series, winning team, losing team, year, winning team won Boolean)

In [6]:
sql_series = 'INSERT INTO Series (postseason_series, winning_team, losing_team, year, WTW) VALUES (?, ?, ?, ?, ?)'
val_series = []
counter = 0
for i in range(len(winning_team_array)):
    val_series.append((postseason_series[i], MLB_functions.read_from_db(winning_team_array[i]), MLB_functions.read_from_db(losing_team_array[i]),year[i],more_wins[i]))
    counter += 1
cur.executemany(sql_series,val_series)

cur.execute('SELECT * FROM Series')
results = cur.fetchall()
print(results)

[('2019 World Series', 30, 11, 2019, 1), ('2019 ALCS', 11, 19, 2019, 1), ('2019 NLCS', 30, 26, 2019, 1), ('2019 ALDS1', 11, 27, 2019, 0), ('2019 ALDS2', 19, 17, 2019, 1), ('2019 NLDS1', 30, 14, 2019, 1), ('2019 NLDS2', 26, 2, 2019, 0), ('2019 ALWC', 27, 20, 2019, 0), ('2019 NLWC', 30, 16, 2019, 1), ('2018 World Series', 4, 14, 2018, 0), ('2018 ALCS', 4, 11, 2018, 1), ('2018 NLCS', 14, 16, 2018, 0), ('2018 ALDS1', 4, 19, 2018, 1), ('2018 ALDS2', 11, 8, 2018, 0), ('2018 NLDS1', 16, 9, 2018, 1), ('2018 NLDS2', 14, 2, 2018, 1), ('2018 ALWC', 19, 20, 2018, 0), ('2018 NLWC', 9, 5, 2018, 0), ('2017 World Series', 11, 14, 2017, 0), ('2017 ALCS', 11, 19, 2017, 0), ('2017 NLCS', 14, 5, 2017, 0), ('2017 ALDS1', 19, 8, 2017, 1), ('2017 ALDS2', 11, 4, 2017, 0), ('2017 NLDS1', 14, 1, 2017, 0), ('2017 NLDS2', 5, 30, 2017, 0), ('2017 ALWC', 19, 17, 2017, 1), ('2017 NLWC', 1, 9, 2017, 1), ('2016 World Series', 5, 8, 2016, 0), ('2016 ALCS', 8, 29, 2016, 1), ('2016 NLCS', 5, 14, 2016, 0), ('2016 ALDS1', 

### Section 2: Extract team statistics from Fox Sports
##### Collect hitting and pitching stats for each team. Web scrape FOX sports page

Mine data from fox sports. Store the data in the team_stats database, stats table

In [7]:
# Request pitching data from fox sports
my_years = []
my_yr = 2019 # enter the current year or any year that you want to start collecting data from
while my_yr >= min(year):
    my_years.append(my_yr)
    my_yr -= 1

for the_year in my_years:

    r = requests.get(f"https://www.foxsports.com/mlb/team-stats?season={the_year}&category=PITCHING+RATIOS&group=1&time=0")
    soup = BeautifulSoup(r.content, 'lxml')

    # request batting data from fox sports
    r_2 = requests.get(f"https://www.foxsports.com/mlb/team-stats?season={the_year}&category=BATTING&group=1&time=0")
    soup_2 = BeautifulSoup(r_2.content, 'lxml')

    main_div_pitching = soup.findAll('table')[0].findAll('tbody')[0]#.findAll('tr')[1].findAll('td')[0].findAll('span')[1]
    main_table_pitching = soup.findAll('table')[0].findAll('tbody')[0].findAll('tr')

    main_table_batting = soup_2.findAll('table')[0].findAll('tbody')[0].findAll('tr')#[1].findAll('td')[0]
    
    
    team_OBP = []
    team_SLG = []
    team_Name = []
    team_HR = []
    for team in main_table_batting:
        Name = str(team.findAll('td')[0].findAll('span')[1]).split('>')[1].split('<')[0]
        if Name not in team_Name:
            team_Name.append(Name)
        else:
            team_Name.append(Name+"_")
        team_OBP.append(str(team.findAll('td')[15]).split('>')[1].split('<')[0])
        team_SLG.append(str(team.findAll('td')[16]).split('>')[1].split('<')[0])
        team_HR.append(str(team.findAll('td')[8]).split('>')[1].split('<')[0])
    
    team_ERA = []
    team_WHIP = []
    team_SO9 = []
    for team in main_table_pitching:
        team_ERA.append(str(team.findAll('td')[17]).split('>')[1].split('<')[0])
        team_WHIP.append(str(team.findAll('td')[8]).split('>')[1].split('<')[0])
        team_SO9.append(str(team.findAll('td')[12]).split('>')[1].split('<')[0])
            
    sql_stats = 'INSERT INTO Stats (team_id, year, OBP, SLG, HR, WHIP, ERA, SO9) VALUES (?, ?, ?, ?, ?, ?, ?, ?)'
    val_stats = []
  
    for i in range(len(team_ERA)):
        val_stats.append((read_from_db(team_Name[i]),the_year,float(team_OBP[i]), float(team_SLG[i]),float(team_HR[i]),float(team_WHIP[i]),float(team_ERA[i]),float(team_SO9[i])))

    cur.executemany(sql_stats,val_stats)

Print the results if visualization is required

In [8]:
cur.execute('SELECT * FROM Stats')
results = cur.fetchall()
for row in results:
    print(row)

(11, 2019, 0.352, 0.495, 288.0, 1.49, 5.56, 7.9)
(17, 2019, 0.338, 0.494, 307.0, 1.46, 5.24, 8.6)
(4, 2019, 0.34, 0.466, 245.0, 1.46, 5.6, 7.8)
(18, 2019, 0.339, 0.49, 306.0, 1.48, 5.2, 7.8)
(30, 2019, 0.342, 0.454, 231.0, 1.46, 5.06, 8.6)
(9, 2019, 0.326, 0.456, 224.0, 1.45, 5.18, 9.0)
(22, 2019, 0.321, 0.42, 163.0, 1.38, 4.99, 7.7)
(6, 2019, 0.314, 0.414, 182.0, 1.3, 4.18, 9.0)
(2, 2019, 0.336, 0.452, 249.0, 1.37, 4.53, 8.6)
(19, 2019, 0.328, 0.442, 242.0, 1.43, 4.79, 8.3)
(13, 2019, 0.338, 0.472, 279.0, 1.43, 4.9, 8.4)
(27, 2019, 0.325, 0.431, 217.0, 1.38, 4.7, 10.0)
(1, 2019, 0.323, 0.434, 220.0, 1.36, 4.19, 8.6)
(5, 2019, 0.331, 0.452, 256.0, 1.38, 5.12, 8.8)
(8, 2019, 0.323, 0.432, 223.0, 1.3, 4.24, 9.4)
(20, 2019, 0.327, 0.448, 257.0, 1.31, 4.25, 8.8)
(28, 2019, 0.319, 0.431, 223.0, 1.3, 4.38, 8.4)
(12, 2019, 0.309, 0.401, 162.0, 1.3, 4.6, 9.3)
(14, 2019, 0.324, 0.422, 220.0, 1.32, 4.1, 9.0)
(3, 2019, 0.31, 0.415, 213.0, 1.3, 4.31, 9.6)
(21, 2019, 0.319, 0.427, 215.0, 1.33, 4.4,

### Section 3: Extract data from the API. This is the player data to get total numbers of the final 40 man rosters

In [9]:
cur.execute('SELECT winning_team, losing_team, year FROM Series')
results = cur.fetchall()

year_and_win = []
year_and_lose = []
year_and_win_and_lose = []
for row in results:
    year_and_win.append([row[2],MLB_functions.reading(row[0])])
    year_and_lose.append([row[2],MLB_functions.reading(row[1])])
    
    year_and_win_and_lose.append([row[2],MLB_functions.reading(row[0]),MLB_functions.reading(row[1])])

print(year_and_win_and_lose)

[[2019, 120, 117], [2019, 117, 147], [2019, 120, 138], [2019, 117, 139], [2019, 147, 142], [2019, 120, 119], [2019, 138, 130], [2019, 139, 133], [2019, 120, 158], [2018, 111, 119], [2018, 111, 117], [2018, 119, 158], [2018, 111, 147], [2018, 117, 114], [2018, 158, 115], [2018, 119, 130], [2018, 147, 133], [2018, 115, 112], [2017, 117, 119], [2017, 117, 147], [2017, 119, 112], [2017, 147, 114], [2017, 117, 111], [2017, 119, 109], [2017, 112, 120], [2017, 147, 142], [2017, 109, 115], [2016, 112, 114], [2016, 114, 141], [2016, 112, 119], [2016, 141, 140], [2016, 114, 111], [2016, 112, 137], [2016, 119, 120], [2016, 141, 110], [2016, 137, 121], [2015, 118, 121], [2015, 118, 141], [2015, 121, 112], [2015, 118, 117], [2015, 141, 140], [2015, 112, 138], [2015, 121, 119], [2015, 117, 147], [2015, 112, 134], [2014, 137, 118], [2014, 118, 110], [2014, 137, 138], [2014, 118, 108], [2014, 110, 116]]


The following functions are needed to scrape the data in the final cell of this section.

In [10]:
def playerID(response_roster):
    """
    The following function takes one roster from the MLB API roster_retrieval function as an input
    outputs an array of the player ID's for the input team
    
    """
    
    import re
    global player_id_array
    player_id_string_array = [m.start() for m in re.finditer('player_id', response_roster.text)]
    player_id_array = []
    for val in player_id_string_array:
        val = int(val)
        player_id = response_roster.text[val+12:val+18]
        player_id_array.append(player_id)
    #     print(player_id)
    return player_id_array

In [11]:
def roster_retrieval(year,team):
    """
    The function uses the year and the team ID input to scrape a roster from the MLB API
    The function outputs the roster information
    Use the year_and_win_and_lose array data
    """
    
    response_roster = []
    url = "https://mlb-data.p.rapidapi.com/json/named.roster_team_alltime.bam"

    querystring = {"all_star_sw":"'N'","sort_order":"name_asc","end_season":str(year),"team_id":str(team),"start_season":str(year-1)}

    headers = {
        'x-rapidapi-host': "mlb-data.p.rapidapi.com",
        'x-rapidapi-key': "52d72158damshf14c292f35c4bdap1ec354jsnc179a7e5ec57"
        }
    response_roster.append(requests.request("GET", url, headers=headers, params=querystring)) #response_roster
    return response_roster

In [12]:
def get_player_name(val):
    """
    The function uses a player ID as an input
    Outputs the name of the player associated with that ID
    Iterate through the output of the player_ID function
    """   
    
    url_2 = "https://mlb-data.p.rapidapi.com/json/named.player_info.bam"
    querystring_2 = {"sport_code":"'mlb'","player_id":str(val)}
    
    headers = {
    'x-rapidapi-host': "mlb-data.p.rapidapi.com",
    'x-rapidapi-key': "52d72158damshf14c292f35c4bdap1ec354jsnc179a7e5ec57"
    }
    
    response_2 = requests.request("GET", url_2, headers=headers, params=querystring_2)
    return response_2

In [13]:
def get_plaer_stat(val):
    """
    The function uses the player ID as an input
    Outputs the statistical profile for that player
    Iterate through the output of the player_ID function
    """
    url = "https://mlb-data.p.rapidapi.com/json/named.sport_hitting_tm.bam"
    querystring = {"season":str(year),"player_id":str(val),"league_list_id":"'mlb'","game_type":"'R'"}
    
    headers = {
    'x-rapidapi-host': "mlb-data.p.rapidapi.com",
    'x-rapidapi-key': "52d72158damshf14c292f35c4bdap1ec354jsnc179a7e5ec57"
    }
    
    response = requests.request("GET", url, headers=headers, params=querystring)
    return response

Scrape the API for the data.  The desired data is the total number of RBIs collects by the entire final roster of each team. The Idea for including this is to account for mid season additions. Remember that teams can trade for players in the middile of the season which means the team stats at the end of the year might not accuratly represent the team entering the playoffs. This will also explain the higher total values extracted.
<br>
<br>
WARNING: The below code requires a lot of processing and could take a while to run

In [14]:
start = time.time()
team_total = []
val_rbi = []
# year_and_win_and_lose =[[2019, 120, 117], [2019, 117, 147], [2019, 120, 138], [2019, 117, 139], [2019, 147, 142], [2019, 120, 119], [2019, 138, 130], [2019, 139, 133], [2019, 120, 158], [2018, 111, 119]]
for row in year_and_win_and_lose:
    year = row[0]
    winning_tm = row[1]
    losing_tm = row[2]
    winning_roster = roster_retrieval(year,winning_tm) # Retreive the rosters
    losing_roster = roster_retrieval(year,losing_tm)
    player_id_winning = playerID(winning_roster[0])
    player_id_losing = playerID(winning_roster[0])
    
    team_total_RBI_win = []
    
    for val in player_id_winning:
        
        player_name = get_player_name(val)
        player_stat = get_plaer_stat(val)

        leng = len(player_stat.text.split('rbi'))
        try:
            RBI = player_stat.text.split("rbi")[1].split(',')[0].split(":")[1][1:-1]
        except:
            pass
        else:
            RBI_array = []
            i = len(player_stat.text.split("rbi"))
            while i > 1:
                RBI = player_stat.text.split("rbi")[i-1].split(',')[0].split(":")[1][1:-1]
                i = i - 1
                RBI_array.append(int(RBI))

        team_total_RBI_win.append(sum(RBI_array))
        
    print(f"the total number of RBI in {year} for win team {winning_tm} is {sum(team_total_RBI_win)}")

    team_total_RBI_lose = []
    
    for val in player_id_losing:
        
        player_name = get_player_name(val)
        player_stat = get_plaer_stat(val)

        leng = len(player_stat.text.split('rbi'))
        
        try:
            RBI = player_stat.text.split("rbi")[1].split(',')[0].split(":")[1][1:-1]
        except IndexError:
            pass
        else:
            RBI_array = []
            i = len(player_stat.text.split("rbi"))
            while i > 1:
                RBI = player_stat.text.split("rbi")[i-1].split(',')[0].split(":")[1][1:-1]
                i = i - 1
                RBI_array.append(int(RBI))

        team_total_RBI_lose.append(sum(RBI_array))
        
    print(f"the total number of RBI in {year} for lose team {losing_tm} is {sum(team_total_RBI_lose)}")

    val_rbi.append((int(year),int(winning_tm),int(sum(team_total_RBI_win)),int(losing_tm),int(sum(team_total_RBI_lose))))


end = time.time()
print(end - start)

the total number of RBI in 2019 for win team 120 is 1373
the total number of RBI in 2019 for lose team 117 is 1373
the total number of RBI in 2019 for win team 117 is 1384
the total number of RBI in 2019 for lose team 147 is 1380
the total number of RBI in 2019 for win team 120 is 1373
the total number of RBI in 2019 for lose team 138 is 1373
the total number of RBI in 2019 for win team 117 is 1384
the total number of RBI in 2019 for lose team 139 is 1380
the total number of RBI in 2019 for win team 147 is 1619
the total number of RBI in 2019 for lose team 142 is 1596
the total number of RBI in 2019 for win team 120 is 1373
the total number of RBI in 2019 for lose team 119 is 1373
the total number of RBI in 2019 for win team 138 is 1231
the total number of RBI in 2019 for lose team 130 is 1231
the total number of RBI in 2019 for win team 139 is 1455
the total number of RBI in 2019 for lose team 133 is 1455
the total number of RBI in 2019 for win team 120 is 1373
the total number of RBI

### Section 4: Analysis 1 of 2
##### First analysis will be simply looking at a teams statistics and running a logistic regression using if they won or lost a series as the target. This mean the team that wins the world series will be assigned a target value of 1 for every series that they played. The team that lost will be assigned a target value of 1 for every series they played except the last. This continues for every team in the playoffs. The classifier is now simply "win" or "not win (lose)". First Analysis does not incoorporate API data. This makes it ideal for trouble shooting the data

In [13]:
cur.execute('SELECT * FROM Series')
#  series, winning, losing, year
results = cur.fetchall()
win = []
lose = []
for row in results:
    win.append((row[1],row[3]))
    lose.append((row[2],row[3]))
print(win)


[(30, 2019), (11, 2019), (30, 2019), (11, 2019), (19, 2019), (30, 2019), (26, 2019), (27, 2019), (30, 2019), (4, 2018), (4, 2018), (14, 2018), (4, 2018), (11, 2018), (16, 2018), (14, 2018), (19, 2018), (9, 2018), (11, 2017), (11, 2017), (14, 2017), (19, 2017), (11, 2017), (14, 2017), (5, 2017), (19, 2017), (1, 2017), (5, 2016), (8, 2016), (5, 2016), (29, 2016), (8, 2016), (5, 2016), (14, 2016), (29, 2016), (24, 2016), (12, 2015), (12, 2015), (18, 2015), (12, 2015), (29, 2015), (5, 2015), (18, 2015), (11, 2015), (5, 2015), (24, 2014), (12, 2014), (24, 2014), (12, 2014), (3, 2014), (24, 2014), (26, 2014), (12, 2014), (24, 2014), (4, 2013), (4, 2013), (26, 2013), (4, 2013), (10, 2013), (26, 2013), (14, 2013), (27, 2013), (22, 2013), (24, 2012), (10, 2012), (24, 2012), (19, 2012), (10, 2012), (26, 2012), (24, 2012), (3, 2012), (26, 2012), (26, 2011), (28, 2011), (26, 2011), (10, 2011), (28, 2011), (26, 2011), (16, 2011), (24, 2010), (28, 2010), (24, 2010), (28, 2010), (19, 2010), (21, 2010

In [14]:
def stat_retreival(winner, loser):
    try:
        cmd_line = "SELECT * FROM Stats WHERE team_id = %d and year = %d" % (winner)
        cur.execute(cmd_line)
        results_winner = cur.fetchall()
    except: 
        results_winner = 'NaN'
    
    try:
        cmd_line = "SELECT * FROM Stats WHERE team_id = %d and year = %d" % (loser)
        cur.execute(cmd_line)
        results_loser = cur.fetchall()
    except:
        results_loser = 'NaN'
        
   
    return (results_winner, results_loser)

In [15]:
# team_id, year, OBP, SLG, HR, WHIP, ERA, SO9
# Changed RBI to HR
logistic_reg_array = []
for i in range(len(win)):
#     stat_retreival(win[i], lose[i])
    (w,l) = stat_retreival(win[i], lose[i])
    logistic_reg_array.append(w)
    logistic_reg_array.append(l)
#     logistic_reg_array.append(win[i], lose[i])
print(logistic_reg_array)


[[(30, 2019, 0.342, 0.454, 231.0, 1.46, 5.06, 8.6)], [(11, 2019, 0.352, 0.495, 288.0, 1.49, 5.56, 7.9)], [(11, 2019, 0.352, 0.495, 288.0, 1.49, 5.56, 7.9)], [(19, 2019, 0.328, 0.442, 242.0, 1.43, 4.79, 8.3)], [(30, 2019, 0.342, 0.454, 231.0, 1.46, 5.06, 8.6)], [(26, 2019, 0.322, 0.415, 210.0, 1.29, 4.27, 9.5)], [(11, 2019, 0.352, 0.495, 288.0, 1.49, 5.56, 7.9)], [(27, 2019, 0.325, 0.431, 217.0, 1.38, 4.7, 10.0)], [(19, 2019, 0.328, 0.442, 242.0, 1.43, 4.79, 8.3)], [(17, 2019, 0.338, 0.494, 307.0, 1.46, 5.24, 8.6)], [(30, 2019, 0.342, 0.454, 231.0, 1.46, 5.06, 8.6)], [(14, 2019, 0.324, 0.422, 220.0, 1.32, 4.1, 9.0)], [(26, 2019, 0.322, 0.415, 210.0, 1.29, 4.27, 9.5)], [(2, 2019, 0.336, 0.452, 249.0, 1.37, 4.53, 8.6)], [(27, 2019, 0.325, 0.431, 217.0, 1.38, 4.7, 10.0)], [(20, 2019, 0.327, 0.448, 257.0, 1.31, 4.25, 8.8)], [(30, 2019, 0.342, 0.454, 231.0, 1.46, 5.06, 8.6)], [(16, 2019, 0.329, 0.438, 250.0, 1.24, 3.97, 8.0)], [(4, 2018, 0.339, 0.453, 208.0, 1.5, 5.19, 7.6)], [(14, 2018, 0.3

In [16]:
# team_id, year, OBP, SLG, HR, WHIP, ERA, SO9
blank = []
counter = 0
for row in logistic_reg_array:
    if row == 'NaN':
        blank.append([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,])
    else:
        if counter % 2 == 0:
            blank.append([row[0][0],row[0][1],row[0][2],row[0][3],row[0][4],row[0][5],row[0][6],row[0][7], 1])
        else:
            blank.append([row[0][0],row[0][1],row[0][2],row[0][3],row[0][4],row[0][5],row[0][6],row[0][7], 0])
    counter += 1
#     print(counter)

# put this into a function that takes the training vec size as an input and returns the test and training vec
training_vec = blank[0:40]
test_vec = []
for val in blank:
    if val not in training_vec:
        test_vec.append(val)
        

Putting the data into a dataframe offers two key advantages: 1) it allows us to clearly see and manipulate the data. 2) It is very compatable with performing the logistic regression analysis. (Oh and it demonstrates knowledge with pandas!)

In [17]:
# team_id, year, OBP, SLG, HR, WHIP, ERA, SO9

df = pd.DataFrame(np.array(blank), columns=['team', 'year','team OBP','team SLG', 'team HR','team WHIP',
                               'team ERA','team SO9','winLoss'])

df = df.dropna(thresh=1)
df

,team,year,team OBP,team SLG,team HR,team WHIP,team ERA,team SO9,winLoss
0,30.0,2019.0,0.342,0.454,231.0,1.46,5.06,8.6,1.0
1,11.0,2019.0,0.352,0.495,288.0,1.49,5.56,7.9,0.0
2,11.0,2019.0,0.352,0.495,288.0,1.49,5.56,7.9,1.0
3,19.0,2019.0,0.328,0.442,242.0,1.43,4.79,8.3,0.0
4,30.0,2019.0,0.342,0.454,231.0,1.46,5.06,8.6,1.0
5,26.0,2019.0,0.322,0.415,210.0,1.29,4.27,9.5,0.0
6,11.0,2019.0,0.352,0.495,288.0,1.49,5.56,7.9,1.0
7,27.0,2019.0,0.325,0.431,217.0,1.38,4.70,10.0,0.0
8,19.0,2019.0,0.328,0.442,242.0,1.43,4.79,8.3,1.0
9,17.0,2019.0,0.338,0.494,307.0,1.46,5.24,8.6,0.0


In [18]:
# df = pd.DataFrame(np.array(blank), columns=['year', 'team', 'team OBP','team SLG', 'team HR','team WHIP',
#                                'team ERA','team SO9','win Loss'])
# # https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python
x_data = df[['team OBP','team SLG', 'team HR','team WHIP','team ERA','team SO9']]
y_data = df.winLoss
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, random_state = 12)
model = LogisticRegression()
model.fit(x_train, y_train)
prediction = dict()
prediction['Logistic'] = model.predict(x_test)
print('Log: ',accuracy_score(y_test, prediction['Logistic']))
print(model.coef_)
conf_mat_logist = confusion_matrix(y_test, prediction['Logistic'])
print('Logist \r', conf_mat_logist)

Log:  0.6052631578947368
[[ 0.01957134  0.02258503 -0.00818118  0.04505393  0.24148623  0.06734832]]
 [[21 14]
 [16 25]]


C:\Users\Peter Argo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Results: Using inductive bias I assigned a test size of 20% of my data set. Using this, method of analysis, we see an improvement on our intial 50% baseline. Recall that using the 190 maximum sample size showed that approximatly 50% of the time does the team with the better record win the series. Using the features clearly shown in the pandas table, we can predict games at approximatly a 61% accuracy. Pretty Good! However, we completely recognize this is statistically fragile as the test and training set are not as comprhensive as ideal. 

### Section 5: Analysis 2 of 2
##### In an attempt to improve on the results, we readjusted out method of analysis. The initial goal was to use home team performance as our target, however the data sources did not lend itself kindly to retreiving this data. Therefore, I wanted to analyize performance of team with a better record. Our classifier is thus not "Winning team had better record" vs. "Winning Team had worse record" (assigned value of 1 and 0 respectively). For simplistic sake, if the winning team had the same record as the losing team, that instance would be assigned a value of 0.  

In [15]:
cur.execute('SELECT * FROM Series')
#  series, winning, losing, year
results = cur.fetchall()
win = []
lose = []
target = []
for row in results:
    win.append((row[1],row[3]))
    lose.append((row[2],row[3]))
    target.append(row[4])
print(target)
print(win)

[1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[(30, 2019), (11, 2019), (30, 2019), (11, 2019), (19, 2019), (30, 2019), (26, 2019), (27, 2019), (30, 2019), (4, 2018), (4, 2018), (14, 2018), (4, 2018), (11, 2018), (16, 2018), (14, 2018), (19, 2018), (9, 2018), (11, 2017), (11, 2017), (14, 2017), (19, 2017), (11, 2017), (14, 2017), (5, 2017), (19, 2017), (1, 2017), (5, 2016), (8, 2016), (5, 2016), (29, 2016), (8, 2016), (5, 2016), (14, 2016), (29, 2016), (24, 2016), (12, 2015), (12, 2015), (18, 2015), (12, 2015), (29, 2015), (5, 2015), (18, 2015), (11, 2015), (5, 2015), (24, 2014), (12, 2014), (24, 2014), (12, 2014), (3, 2014)]


In [16]:
def stat_retreival(winner, loser):
    try:
        cmd_line = "SELECT * FROM Stats WHERE team_id = %d and year = %d" % (winner)
        cur.execute(cmd_line)
        results_winner = cur.fetchall()
    except: 
        results_winner = 'NaN'
    
    try:
        cmd_line = "SELECT * FROM Stats WHERE team_id = %d and year = %d" % (loser)
        cur.execute(cmd_line)
        results_loser = cur.fetchall()
    except:
        results_loser = 'NaN'
        
   
    return (results_winner, results_loser)

In [17]:
logistic_reg_array2 = []
for i in range(len(win)):
    logistic_reg_array2.append([stat_retreival(win[i], lose[i]),target[i]])

The next few cells combines our API data with our fos sports team stat data

In [18]:
new_val_rbi = []
for row in val_rbi:
    cmd_line = "SELECT team_id FROM prim_ID WHERE API_ID = %d" % row[1]
    cur.execute(cmd_line)
    results_winner_flip = cur.fetchall()
    
    cmd_line = "SELECT team_id FROM prim_ID WHERE API_ID = %d" % row[3]
    cur.execute(cmd_line)
    results_loser_flip = cur.fetchall()
    new_val_rbi.append([row[0],results_winner_flip[0][0],row[2],results_loser_flip[0][0],row[4]])

print(new_val_rbi)

[[2019, 30, 1373, 11, 1373], [2019, 11, 1384, 19, 1380], [2019, 30, 1373, 26, 1373], [2019, 11, 1384, 27, 1380], [2019, 19, 1619, 17, 1596], [2019, 30, 1373, 14, 1373], [2019, 26, 1231, 2, 1231], [2019, 27, 1455, 20, 1455], [2019, 30, 1373, 16, 1373], [2018, 4, 1031, 14, 1017], [2018, 4, 1017, 11, 1017], [2018, 14, 1257, 16, 1257], [2018, 4, 1004, 19, 1017], [2018, 11, 1359, 8, 1359], [2018, 16, 1328, 9, 1328], [2018, 14, 1257, 2, 1257], [2018, 19, 1448, 20, 1448], [2018, 9, 980, 5, 1001], [2017, 11, 1190, 14, 1190], [2017, 11, 1190, 19, 1190], [2017, 14, 953, 5, 953], [2017, 19, 1435, 8, 1447], [2017, 11, 1190, 4, 1190], [2017, 14, 953, 1, 953], [2017, 5, 997, 30, 997], [2017, 19, 1481, 17, 1447], [2017, 1, 1251, 9, 1251], [2016, 5, 1385, 8, 1385], [2016, 8, 1380, 29, 1304], [2016, 5, 1385, 14, 1385], [2016, 29, 1387, 28, 1312], [2016, 8, 1355, 4, 1304], [2016, 5, 1385, 24, 1385], [2016, 14, 948, 30, 948], [2016, 29, 1165, 3, 1312], [2016, 24, 998, 18, 998], [2015, 12, 1490, 18, 1572]

In [19]:
logistic_reg_array3 = []
for i in range(len(new_val_rbi)):
    if new_val_rbi[i][1] == logistic_reg_array2[i][0][0][0][0] and new_val_rbi[i][3] == logistic_reg_array2[i][0][1][0][0]:
        logistic_reg_array3.append([logistic_reg_array2[i],new_val_rbi[i][2],new_val_rbi[i][4]])

In [20]:
blank2 = []
counter = 0
for row in logistic_reg_array3:
#     print(row[0][1][0][2])
    try:
        blank2.append([row[0][0][0][0][1],row[0][0][0][0][0],row[0][0][0][0][2],row[0][0][0][0][3],row[0][0][0][0][4],\
                   row[0][0][0][0][5],row[0][0][0][0][6],row[0][0][0][0][7],row[1],row[0][0][1][0][0],row[0][0][1][0][2],\
                   row[0][0][1][0][3],row[0][0][1][0][4],row[0][0][1][0][5],row[0][0][1][0][6],row[0][0][1][0][7],row[2],\
                   row[0][1]])
    except:
        blank2.append([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
    counter += 1
#     print(counter)

print(blank2)

[[2019, 30, 0.342, 0.454, 231.0, 1.46, 5.06, 8.6, 1373, 11, 0.352, 0.495, 288.0, 1.49, 5.56, 7.9, 1373, 1], [2019, 11, 0.352, 0.495, 288.0, 1.49, 5.56, 7.9, 1384, 19, 0.328, 0.442, 242.0, 1.43, 4.79, 8.3, 1380, 1], [2019, 30, 0.342, 0.454, 231.0, 1.46, 5.06, 8.6, 1373, 26, 0.322, 0.415, 210.0, 1.29, 4.27, 9.5, 1373, 1], [2019, 11, 0.352, 0.495, 288.0, 1.49, 5.56, 7.9, 1384, 27, 0.325, 0.431, 217.0, 1.38, 4.7, 10.0, 1380, 0], [2019, 19, 0.328, 0.442, 242.0, 1.43, 4.79, 8.3, 1619, 17, 0.338, 0.494, 307.0, 1.46, 5.24, 8.6, 1596, 1], [2019, 30, 0.342, 0.454, 231.0, 1.46, 5.06, 8.6, 1373, 14, 0.324, 0.422, 220.0, 1.32, 4.1, 9.0, 1373, 1], [2019, 26, 0.322, 0.415, 210.0, 1.29, 4.27, 9.5, 1231, 2, 0.336, 0.452, 249.0, 1.37, 4.53, 8.6, 1231, 0], [2019, 27, 0.325, 0.431, 217.0, 1.38, 4.7, 10.0, 1455, 20, 0.327, 0.448, 257.0, 1.31, 4.25, 8.8, 1455, 0], [2019, 30, 0.342, 0.454, 231.0, 1.46, 5.06, 8.6, 1373, 16, 0.329, 0.438, 250.0, 1.24, 3.97, 8.0, 1373, 1], [2018, 4, 0.339, 0.453, 208.0, 1.5, 5.

In [21]:
df2 = pd.DataFrame(np.array(blank2), columns=['year', 'winning team', 'winning team OBP','winning team SLG', 'winning team HR','winning team WHIP',
                               'winning team ERA','winning team SO9','winning team rbi','losing team', 'losing team OBP','losing team SLG', 'losing team HR','losing team WHIP',
                               'losing team ERA','losing team SO9','losing team rbi','Target'])
df2['diff OBP'] = df2['winning team OBP'] - df2['losing team OBP']
df2['diff SLG'] = df2['winning team SLG'] - df2['losing team SLG']
df2['diff HR'] = df2['winning team HR'] - df2['losing team HR']
df2['diff WHIP'] = df2['winning team WHIP'] - df2['losing team WHIP']
df2['diff ERA'] = df2['winning team ERA'] - df2['losing team ERA']
df2['diff SO9'] = df2['winning team SO9'] - df2['losing team SO9']
df2['diff RBI'] = df2['winning team rbi'] - df2['losing team rbi']
df2 = df2.dropna(thresh=1)

df2

,year,winning team,winning team OBP,winning team SLG,winning team HR,winning team WHIP,winning team ERA,winning team SO9,winning team rbi,losing team,...,losing team SO9,losing team rbi,Target,diff OBP,diff SLG,diff HR,diff WHIP,diff ERA,diff SO9,diff RBI
0,2019.0,30.0,0.342,0.454,231.0,1.46,5.06,8.6,1373.0,11.0,...,7.9,1373.0,1.0,-0.010,-0.041,-57.0,-0.03,-0.50,0.7,0.0
1,2019.0,11.0,0.352,0.495,288.0,1.49,5.56,7.9,1384.0,19.0,...,8.3,1380.0,1.0,0.024,0.053,46.0,0.06,0.77,-0.4,4.0
2,2019.0,30.0,0.342,0.454,231.0,1.46,5.06,8.6,1373.0,26.0,...,9.5,1373.0,1.0,0.020,0.039,21.0,0.17,0.79,-0.9,0.0
3,2019.0,11.0,0.352,0.495,288.0,1.49,5.56,7.9,1384.0,27.0,...,10.0,1380.0,0.0,0.027,0.064,71.0,0.11,0.86,-2.1,4.0
4,2019.0,19.0,0.328,0.442,242.0,1.43,4.79,8.3,1619.0,17.0,...,8.6,1596.0,1.0,-0.010,-0.052,-65.0,-0.03,-0.45,-0.3,23.0
5,2019.0,30.0,0.342,0.454,231.0,1.46,5.06,8.6,1373.0,14.0,...,9.0,1373.0,1.0,0.018,0.032,11.0,0.14,0.96,-0.4,0.0
6,2019.0,26.0,0.322,0.415,210.0,1.29,4.27,9.5,1231.0,2.0,...,8.6,1231.0,0.0,-0.014,-0.037,-39.0,-0.08,-0.26,0.9,0.0
7,2019.0,27.0,0.325,0.431,217.0,1.38,4.70,10.0,1455.0,20.0,...,8.8,1455.0,0.0,-0.002,-0.017,-40.0,0.07,0.45,1.2,0.0
8,2019.0,30.0,0.342,0.454,231.0,1.46,5.06,8.6,1373.0,16.0,...,8.0,1373.0,1.0,0.013,0.016,-19.0,0.22,1.09,0.6,0.0
9,2018.0,4.0,0.339,0.453,208.0,1.50,5.19,7.6,1031.0,14.0,...,8.8,1017.0,0.0,0.026,0.040,-6.0,0.25,1.15,-1.2,14.0


In [29]:
x_data = df2[['diff OBP','diff SLG', 'diff HR','diff WHIP','diff ERA','diff SO9','diff RBI']]
y_data = df2.Target
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, random_state = 12)
model = LogisticRegression()
model.fit(x_train, y_train)
prediction = dict()
prediction['Logistic'] = model.predict(x_test)
print('Log: ',accuracy_score(y_test, prediction['Logistic']))
print(model.coef_)
conf_mat_logist = confusion_matrix(y_test, prediction['Logistic'])
print('Logist \r', conf_mat_logist)

Log:  0.5333333333333333
[[ 0.02186673  0.00377467 -0.01345052  0.1574246   0.17640326 -0.4685162
   0.00179922]]
 [[5 2]
 [5 3]]


C:\Users\Peter Argo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Results: I increased the test size partition because we are working with 50 samples as opposed to 190 as in analysis 1.  We see a 53% prediction accuracy however nothing can be concluded with only 50 samples.

# --------------------------------------------------------------------------------------------------------------